# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,13.89,90,100,9.78,SH,2023-12-12
1,1,adamstown,-25.0660,-130.1015,21.68,60,5,1.02,PN,2023-12-12
2,2,aasiaat,68.7098,-52.8699,-0.78,78,100,5.74,GL,2023-12-12
3,3,ribeira grande,38.5167,-28.7000,13.76,81,7,2.68,PT,2023-12-12
4,4,sainte-marie,-20.8968,55.5496,23.09,84,40,2.06,RE,2023-12-12


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)
# Display the map
# YOUR CODE HERE
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()


12


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,floro,61.5996,5.0328,5.54,64,5,4.63,NO,2023-12-12
46,46,papatowai,-46.5619,169.4708,10.76,61,40,9.12,NZ,2023-12-12
48,48,punta arenas,-53.1500,-70.9167,11.73,46,77,7.77,CL,2023-12-12
200,200,invercargill,-46.4000,168.3500,11.62,51,44,9.31,NZ,2023-12-12
270,270,zahedan,29.4963,60.8629,3.95,45,0,3.09,IR,2023-12-12


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
29,floro,NO,61.5996,5.0328,64,
46,papatowai,NZ,-46.5619,169.4708,61,
48,punta arenas,CL,-53.1500,-70.9167,46,
200,invercargill,NZ,-46.4000,168.3500,51,
270,zahedan,IR,29.4963,60.8629,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
floro - nearest hotel: Comfort Hotel Victoria Florø
papatowai - nearest hotel: No hotel found
punta arenas - nearest hotel: Hotel Lacolet
invercargill - nearest hotel: The Grand
zahedan - nearest hotel: هتل صالح;بانک ملی
chinde - nearest hotel: Kanyum cottage village
battagram - nearest hotel: No hotel found
kitimat - nearest hotel: MStar Hotel
yacimiento rio turbio - nearest hotel: Capipe
najran - nearest hotel: No hotel found
gerash - nearest hotel: هتل آبشار
dubrovnik - nearest hotel: Bellevue


,City,Country,Lat,Lng,Humidity,Hotel Name
29,floro,NO,61.5996,5.0328,64,Comfort Hotel Victoria Florø
46,papatowai,NZ,-46.5619,169.4708,61,No hotel found
48,punta arenas,CL,-53.1500,-70.9167,46,Hotel Lacolet
200,invercargill,NZ,-46.4000,168.3500,51,The Grand
270,zahedan,IR,29.4963,60.8629,45,هتل صالح;بانک ملی
410,chinde,NP,26.8418,88.0763,61,Kanyum cottage village
422,battagram,PK,34.6772,73.0233,34,No hotel found
443,kitimat,CA,54.0524,-128.6534,65,MStar Hotel
451,yacimiento rio turbio,AR,-51.5732,-72.3508,68,Capipe
470,najran,SA,17.4924,44.1277,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)
# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)